In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
DATA_DIR = "data/"
FILE_NAME = "data.csv"
FINAL_DATA = "rearranged_data.xlsx"
DATA_SPECS = "data_specs.json"

In [3]:
with open(DATA_SPECS, 'r') as f:
    DATA_SPECS_DICT = json.load(f)

In [4]:
# Load data
df = pd.read_csv(os.path.join(DATA_DIR, FILE_NAME), delimiter=";")

/Users/ben.glaeser/.venvs/ir/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# function to copy serial
def copy_serial(row):
    if not pd.isnull(row["ZG04"]):
        row["SERIAL"] = row["ZG04"]
    elif not pd.isnull(row["ZG05"]):
        row["SERIAL"] = row["ZG05"]
    return row

In [6]:
# move serial to serial from w01
df = df.apply(lambda row: copy_serial(row), axis=1)

In [7]:
# Drop lines where we have no serial number
df = df[~pd.isnull(df["SERIAL"])]

In [8]:
# Function to extract group
serial_group = dict()
def extract_variable(row):
    if not pd.isnull(row["ZG04"]):
        serial_group.update({row["SERIAL"]:"MS"})
    elif not pd.isnull(row["ZG05"]):
        serial_group.update({row["SERIAL"]:"AL"})

In [9]:
%%capture
# Extract group
df.apply(lambda row: extract_variable(row), axis=1)

In [10]:
# Drop some unnecessary columns
df.drop(DATA_SPECS_DICT["drop_vars"], axis=1, inplace=True)

In [11]:
# Find all cases that have completed all the stuff
def collect_complete_cases():
    complete_cases = []
    for indiv in df["SERIAL"].unique():
        df_indiv = df[df["SERIAL"]== indiv]
        if df_indiv.shape[0] > 9:
            questionnaires = df_indiv["QUESTNNR"].values
            if ("MS_10" in questionnaires or "Altern10" in questionnaires) and "A2" in questionnaires and "wi01" in questionnaires:
                complete_cases.append(indiv)
    return complete_cases

complete_cases = collect_complete_cases()
df = df[df["SERIAL"].isin(complete_cases)]

In [12]:
value_vars_drop = ['SERIAL',"QUESTNNR"]
value_vars = [x for x in df.columns if x not in value_vars_drop]
df = pd.melt(df, id_vars=["SERIAL","QUESTNNR"], value_vars=value_vars)

In [13]:
# Drop variables without anser
df = df[~pd.isnull(df["value"])]

In [14]:
# Add group variable
df["GROUP"] = df["SERIAL"].apply(lambda val: serial_group[val])

In [15]:
# Rearrange table
df = df[["SERIAL", "GROUP", "QUESTNNR", "variable", "value"]]

In [17]:
# Store to new excel
df.to_excel(os.path.join(DATA_DIR, FINAL_DATA), index=False)

In [16]:
df["SERIAL"].value_counts()

4YVK3TVP56    12
X8MWUXUUV9    12
7C5MWUEWVU    12
YR9U3RQQFK    12
CE2CQ4676P    12
95WDLQHH5X    12
66XVE74VSZ    12
ZFZGH15H3R    12
Y54LCLTAB1    12
MK2NX77GP2    12
4HHS5UVCQX    12
KR8GDNPVQV    12
VGLSKZHF76    12
HGQ5AY46N4    12
NZPC4NZA87    12
YQVLR33DTR    12
81BK12ZEZM    12
VDFEG5D3RY    12
S6HZHHP2T7    12
E2EZXNPQS7    12
YVGY71FZB8    12
ZPRF8XRA73    12
RV3WBGXP3U    12
6R9BZS8RTL    12
DUGRETE4D2    12
4BC5A2C2ND    12
UW2R67HH3N    12
31XD4UFDA7    12
NMFG4N6PZA    12
NUCWLGXQNG    12
              ..
1WTHKTGXN5     1
ZFVAL2V7HR     1
V7LLTQ8W96     1
PX8NZU9UC7     1
Q3M46CYUPR     1
FRDLHF4EPW     1
MEDEMHG7CS     1
S7F7X99QAC     1
WWB2XAB2C3     1
HFNQ34EXCC     1
V9H4SSHWLT     1
LMFVNVTQR3     1
VMB22VYKES     1
RT49GDWTG3     1
E3T8YF9M15     1
U5QAUR9CPP     1
K6L3H4C8NN     1
DCCPX86A12     1
PW75ZB6C86     1
84PD6YP9RN     1
SY4RXBK75B     1
LTNDN5U27X     1
R9KGU85HT3     1
KDC54ELWPR     1
A3QUY36GEX     1
DHQ4B6Q3CL     1
23H5AQF5NB     1
ZACR1PGQ49    

In [12]:
df.head()

,SERIAL,QUESTNNR,SD01,SD02_01,SD19,SD20,SD21,SD10,SD10_09,SD22,...,HA21_11,HA21_12,HA21_13,HA21_14,HA21_15,HA21_16,HA21_17,HA21_18,ZG01_CP,ZG01
0,DA1A1CDBFR,wi01,2.0,29.0,2.0,1.0,5.0,4.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0
1,DA1A1CDBFR,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,NaN,NaN
2,K9E9XH4MVB,wi01,2.0,29.0,1.0,1.0,2.0,7.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
3,3FDC1GTMB9,wi01,2.0,29.0,1.0,1.0,2.0,8.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,K9E9XH4MVB,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,NaN,NaN
